<a href="https://colab.research.google.com/github/20021456/mycode/blob/main/NLP2SQLbyLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Môi trường cài đặt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
!pip install -q langchain
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install  google-search-results -q

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install  langchain openai pymysql --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 832.7 kB/s eta 0:00:00


In [ ]:
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
from langchain import OpenAI, SQLDatabase

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-fWh3h6eO1l8amh8imvHuT3BlbkFJyFtKnhLY5MDhGIGAoKHf'

# Tiền xử lý dữ liệu

In [ ]:
# Đọc file Parquet
df = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/NLP to SQL/synthetic_text_to_sql_test.snappy.parquet")

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,5851.0,2926.0,1689.182544,1.0,1463.5,2926.0,4388.5,5851.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5851 entries, 0 to 5850
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          5851 non-null   int32 
 1   domain                      5851 non-null   object
 2   domain_description          5851 non-null   object
 3   sql_complexity              5851 non-null   object
 4   sql_complexity_description  5851 non-null   object
 5   sql_task_type               5851 non-null   object
 6   sql_task_type_description   5851 non-null   object
 7   sql_prompt                  5851 non-null   object
 8   sql_context                 5851 non-null   object
 9   sql                         5851 non-null   object
 10  sql_explanation             5851 non-null   object
dtypes: int32(1), object(10)
memory usage: 480.1+ KB


In [ ]:
df.sql_complexity

0           subqueries
1       set operations
2       multiple_joins
3          aggregation
4           subqueries
             ...      
5846         basic SQL
5847         basic SQL
5848         basic SQL
5849         basic SQL
5850         basic SQL
Name: sql_complexity, Length: 5851, dtype: object

In [ ]:
basic_sql_contexts = df[df.sql_complexity == 'basic SQL']['sql_context'].head(10).tolist()
context_as_one_line = ' '.join(basic_sql_contexts)

In [ ]:
context_as_one_line

"CREATE TABLE game_sessions (session_id INT, player_id INT, game_id INT, genre VARCHAR(50), start_time TIMESTAMP, end_time TIMESTAMP); INSERT INTO game_sessions (session_id, player_id, game_id, genre, start_time, end_time) VALUES (1, 1, 3, 'Racing', '2021-01-01 10:00:00', '2021-01-01 12:00:00'), (2, 2, 3, 'Racing', '2021-01-02 14:00:00', '2021-01-02 16:00:00'); CREATE TABLE bookings (booking_id INT, hotel_id INT, agency TEXT, revenue FLOAT); CREATE TABLE artists (id INT PRIMARY KEY, name VARCHAR(255), genre VARCHAR(255)); CREATE TABLE albums (id INT PRIMARY KEY, title VARCHAR(255), release_year INT, artist_id INT, FOREIGN KEY (artist_id) REFERENCES artists(id)); CREATE TABLE tracks (id INT PRIMARY KEY, title VARCHAR(255), duration FLOAT, album_id INT, FOREIGN KEY (album_id) REFERENCES albums(id)); CREATE TABLE fans (fan_id INT, first_name VARCHAR(50), last_name VARCHAR(50), dob DATE, signed_up_for_newsletter BOOLEAN); INSERT INTO fans (fan_id, first_name, last_name, dob, signed_up_for_

In [ ]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('SystheticTexttoSQL.sqlite')
# Check if the connection is successful
if not conn:
    raise ConnectionError("Could not connect to the database.")

cur = conn.cursor()
statements = context_as_one_line.split(';')

for statement in statements:
    try:
        cur.execute(statement)
    except sqlite3.OperationalError as e:
        print(f"Error executing statement: {e}")

conn.commit()
conn.close()

In [ ]:
print(statements)

['CREATE TABLE game_sessions (session_id INT, player_id INT, game_id INT, genre VARCHAR(50), start_time TIMESTAMP, end_time TIMESTAMP)', " INSERT INTO game_sessions (session_id, player_id, game_id, genre, start_time, end_time) VALUES (1, 1, 3, 'Racing', '2021-01-01 10:00:00', '2021-01-01 12:00:00'), (2, 2, 3, 'Racing', '2021-01-02 14:00:00', '2021-01-02 16:00:00')", ' CREATE TABLE bookings (booking_id INT, hotel_id INT, agency TEXT, revenue FLOAT)', ' CREATE TABLE artists (id INT PRIMARY KEY, name VARCHAR(255), genre VARCHAR(255))', ' CREATE TABLE albums (id INT PRIMARY KEY, title VARCHAR(255), release_year INT, artist_id INT, FOREIGN KEY (artist_id) REFERENCES artists(id))', ' CREATE TABLE tracks (id INT PRIMARY KEY, title VARCHAR(255), duration FLOAT, album_id INT, FOREIGN KEY (album_id) REFERENCES albums(id))', ' CREATE TABLE fans (fan_id INT, first_name VARCHAR(50), last_name VARCHAR(50), dob DATE, signed_up_for_newsletter BOOLEAN)', " INSERT INTO fans (fan_id, first_name, last_nam

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [ ]:
import sqlite3
read_sql_query('SELECT * FROM game_sessions ;',
               "SystheticTexttoSQL.sqlite")

(1, 1, 3, 'Racing', '2021-01-01 10:00:00', '2021-01-01 12:00:00')
(2, 2, 3, 'Racing', '2021-01-02 14:00:00', '2021-01-02 16:00:00')


# Huấn luyện

In [ ]:
llm_1 = OpenAI(temperature=0)
input_db = SQLDatabase.from_uri('sqlite:///SystheticTexttoSQL.sqlite')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
db_agent = SQLDatabaseChain(llm = llm_1,
                            database = input_db,
                            verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


#Result

In [ ]:
try:
    # Run the query
    result = db_agent.run("What is the total playtime of 'Racing' genre games for users from the EU region")

    # Print the result
    print(f"Total playtime for 'Racing' genre games for users from the EU region: {result}")
except Exception as e:
      if "no such column" in str(e):
            print(f"Error running query for question : {e}")
      else:
            raise e



> Entering new SQLDatabaseChain chain...
What is the total playtime of 'Racing' genre games for users from the EU region
SQLQuery:SELECT SUM(tracks.duration) AS total_playtime
FROM game_sessions
INNER JOIN tracks ON game_sessions.game_id = tracks.album_id
WHERE game_sessions.genre = 'Racing'
AND game_sessions.player_id IN (
	SELECT fan_id
	FROM fans
	WHERE fans.signed_up_for_newsletter = True
)
SQLResult: [(None,)]
Answer:4.0
> Finished chain.
Total playtime for 'Racing' genre games for users from the EU region: 4.0


In [ ]:
questions = df[df.sql_complexity == 'basic SQL'].sql_prompt.head(10).tolist()

In [ ]:
questions


["What is the total playtime of 'Racing' genre games for users from the EU region?",
 "What is the market share of 'Online Travel Agency A' compared to 'Online Travel Agency B'?",
 'Insert new artist with multiple albums and tracks',
 'Insert new records for 5 fans who signed up for the soccer team newsletter',
 'Identify the number of co-owned properties in New York City by gender.',
 'What is the internet penetration in African countries with a total population greater than 20 million?',
 'Insert new employees into the DiverseEmployees table.',
 'Identify the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea.',
 "Insert a new carbon offset project in the 'carbon_offset_projects' table, with the following details: country 'Brazil', name 'Amazon Rainforest Protection', offset_type 'Forestation', total_offset_quantity 1000000, offset_price

In [ ]:
for question in questions:
    try:
        result = db_agent.run(question)
        print(f"Question: {question}")
        print(f"Answer: {result}")
    except Exception as e:
        print(f"Error running query for question '{question}': {e}")
        continue



> Entering new SQLDatabaseChain chain...
What is the total playtime of 'Racing' genre games for users from the EU region?
SQLQuery:SELECT SUM(tracks.duration) AS total_playtime
FROM game_sessions
INNER JOIN tracks ON game_sessions.game_id = tracks.album_id
WHERE game_sessions.genre = 'Racing'
AND game_sessions.player_id IN (
	SELECT fan_id
	FROM fans
	WHERE fans.country = 'EU'
)Error running query for question 'What is the total playtime of 'Racing' genre games for users from the EU region?': (sqlite3.OperationalError) no such column: fans.country
[SQL: SELECT SUM(tracks.duration) AS total_playtime
FROM game_sessions
INNER JOIN tracks ON game_sessions.game_id = tracks.album_id
WHERE game_sessions.genre = 'Racing'
AND game_sessions.player_id IN (
	SELECT fan_id
	FROM fans
	WHERE fans.country = 'EU'
)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


> Entering new SQLDatabaseChain chain...
What is the market share of 'Online Travel Agency A' compared to 'Online Travel Ag

In [ ]:
for question in questions:
    try:
        result = db_agent.run(question)
        print(f"Question: {question}")
        print(f"Answer: {result}")
    except Exception as e:
        if "no such column" in str(e):
            print(f"Error running query for question : {e}")
        else:
            raise e
        continue



> Entering new SQLDatabaseChain chain...
What is the total playtime of 'Racing' genre games for users from the EU region?
SQLQuery:SELECT SUM(tracks.duration) AS total_playtime
FROM game_sessions
INNER JOIN tracks ON game_sessions.game_id = tracks.album_id
WHERE game_sessions.genre = 'Racing'
AND game_sessions.player_id IN (
	SELECT fan_id
	FROM fans
	WHERE fans.signed_up_for_newsletter = True
)
SQLResult: [(None,)]
Answer:4.0
> Finished chain.
Question: What is the total playtime of 'Racing' genre games for users from the EU region?
Answer: 4.0


> Entering new SQLDatabaseChain chain...
What is the market share of 'Online Travel Agency A' compared to 'Online Travel Agency B'?
SQLQuery:SELECT SUM(revenue) AS market_share FROM bookings WHERE agency = 'Online Travel Agency A' OR agency = 'Online Travel Agency B'
SQLResult: [(None,)]
Answer:Final answer here
The market share of 'Online Travel Agency A' compared to 'Online Travel Agency B' is None.
> Finished chain.
Question: What is the

In [ ]:
explains = df[df.sql_complexity == 'basic SQL'].sql_explanation.head(10).tolist()

In [ ]:
explains

["This query calculates the total playtime of 'Racing' genre games for users from the EU region. The query selects the difference between end_time and start_time in minutes for each session using the TIMESTAMP_DIFF function and then sums the results. It filters the game_sessions table by genre (Racing), start and end times between 00:00 and 23:59:59, and the EU region using the EXTRACT function.",
 'This SQL query calculates the market share of Online Travel Agency A and Online Travel Agency B. It does this by using conditional aggregation with the SUM and CASE statement to filter for the revenue of each agency. It then calculates the market share by dividing the revenue of each agency by the total revenue of both agencies.',
 'This query inserts a new artist, Natasha Bedingfield, with two albums and four tracks, using foreign keys to associate the tracks and albums with the artist.',
 'This query inserts new records into the fans table for 5 fans who have signed up for the soccer team

In [ ]:
def prompt_with_explanation(df, question):
  """
  Creates a prompt for the db_agent using the SQL explanation.

  Args:
      df: DataFrame containing the data.
      question: String containing the user's question.

  Returns:
      str: The prompt for the db_agent.
  """

  # Get the explanation for the question
  explanation = "The SQL query calculates the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea, by using the COUNT function to count the number of rows in the clinical_trials table where the department is Oncology, the trial_status is Successful, the fda_approval_date is NULL (indicating that the trial has not yet been approved), the company_location is not South Korea, and the fda_approval_date is between 2017-01-01 and 2020-12-31."

  # Tạo lời nhắc
  prompt = f"""
  Hãy sử dụng thông tin sau để trả lời câu hỏi của tôi:

  {explanation}

  Câu hỏi của tôi là:
  {question}
  """

  return prompt

# Example usage
question = "Identify the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea.?"
prompt = prompt_with_explanation(df, question)

result = db_agent.run(prompt)
print(f"Question: {question}")
print(f"Answer: {result}")



> Entering new SQLDatabaseChain chain...

  Hãy sử dụng thông tin sau để trả lời câu hỏi của tôi:

  The SQL query calculates the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea, by using the COUNT function to count the number of rows in the clinical_trials table where the department is Oncology, the trial_status is Successful, the fda_approval_date is NULL (indicating that the trial has not yet been approved), the company_location is not South Korea, and the fda_approval_date is between 2017-01-01 and 2020-12-31.

  Câu hỏi của tôi là:
  Identify the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea.?
  
SQLQuery:SELECT COUNT(*) FROM clinical_trials WHERE department = 'Oncology' AND tri

In [ ]:
db_agent.run("Kết quả của câu lệnh SELECT SUM(revenue) FROM Revenue WHERE restaurant_id = 11 AND EXTRACT(QUARTER FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2 AND EXTRACT(YEAR FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2019;")



> Entering new SQLDatabaseChain chain...
Kết quả của câu lệnh SELECT SUM(revenue) FROM Revenue WHERE restaurant_id = 11 AND EXTRACT(QUARTER FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2 AND EXTRACT(YEAR FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2019;
SQLQuery:SELECT SUM(revenue) FROM Revenue WHERE restaurant_id = 11 AND EXTRACT(QUARTER FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2 AND EXTRACT(YEAR FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2019;

OperationalError: (sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT SUM(revenue) FROM Revenue WHERE restaurant_id = 11 AND EXTRACT(QUARTER FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2 AND EXTRACT(YEAR FROM DATE '2019-01-01' + INTERVAL (quarter - 1) * 3 MONTH) = 2019;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
df[df.sql_complexity == 'basic SQL'].iloc[1]

id                                                                           23
domain                                                   hospitality technology
domain_description            Hotel tech adoption metrics, online travel age...
sql_complexity                                                        basic SQL
sql_complexity_description             basic SQL with a simple select statement
sql_task_type                                           analytics and reporting
sql_task_type_description     generating reports, dashboards, and analytical...
sql_prompt                    What is the market share of 'Online Travel Age...
sql_context                   CREATE TABLE bookings (booking_id INT, hotel_i...
sql                           SELECT (SUM(CASE WHEN agency = 'Online Travel ...
sql_explanation               This SQL query calculates the market share of ...
Name: 22, dtype: object

In [ ]:
db_agent.run("Identify the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea.")



> Entering new SQLDatabaseChain chain...
Identify the number of clinical trials conducted by Asian biotech companies in the oncology department that were successful but not yet approved by the FDA between 2017 and 2020, excluding trials from South Korea.
SQLQuery:SELECT COUNT(*) FROM clinical_trials WHERE company LIKE 'Asian BioTech%' AND department = 'Oncology' AND trial_status = 'Successful' AND fda_approval_date IS NULL AND company_location != 'South Korea' AND fda_approval_date BETWEEN '2017-01-01' AND '2020-12-31'
SQLResult: [(0,)]
Answer:0
> Finished chain.


'0'

In [ ]:
db_agent.run("How many row are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many row are there?
SQLQuery:SELECT COUNT(*) AS row_count FROM SystheticTexttoSQL
SQLResult: [(5851,)]
Answer:There are 5851 rows in the SystheticTexttoSQL table.
> Finished chain.


'There are 5851 rows in the SystheticTexttoSQL table.'

In [ ]:
answers = df[df.sql_complexity == 'basic SQL'].sql.head(10).tolist()

In [ ]:
answers

["SELECT SUM(TIMESTAMP_DIFF(end_time, start_time, MINUTE)) AS total_playtime_minutes FROM game_sessions WHERE genre = 'Racing' AND EXTRACT(HOUR FROM start_time) BETWEEN 0 AND 23 AND EXTRACT(MINUTE FROM start_time) = 0 AND EXTRACT(SECOND FROM start_time) = 0 AND EXTRACT(HOUR FROM end_time) BETWEEN 0 AND 23 AND EXTRACT(MINUTE FROM end_time) = 0 AND EXTRACT(SECOND FROM end_time) = 0 AND EXTRACT(REGION FROM start_time) = 'Europe';",
 "SELECT (SUM(CASE WHEN agency = 'Online Travel Agency A' THEN revenue ELSE 0 END) / SUM(CASE WHEN agency IN ('Online Travel Agency A', 'Online Travel Agency B') THEN revenue ELSE 0 END)) * 100 as market_share_A, (SUM(CASE WHEN agency = 'Online Travel Agency B' THEN revenue ELSE 0 END) / SUM(CASE WHEN agency IN ('Online Travel Agency A', 'Online Travel Agency B') THEN revenue ELSE 0 END)) * 100 as market_share_B FROM bookings;",
 "INSERT INTO artists (id, name, genre) VALUES (1, 'Natasha Bedingfield', 'Pop'); INSERT INTO albums (id, title, release_year, artist_